In [8]:
'''
pip install folium
pip install geopandas
pip install geotable
pip install --upgrade shapely numpy
in
pip install --upgrade pyproj
'''

import pandas as pd
import numpy as np
import geopandas as gpd
# import geotable
from pyproj import Proj, transform
# from shapely.geometry import Point
from shapely.geometry import Point, LineString, Polygon
import folium

# from google.colab import drive
# drive.mount('/content/drive')
df_point = pd.read_csv('./정박지 관련 데이터/ulsan_geo_points_final.csv')


df_point = df_point.drop(columns=['지점'])
M1 = df_point.loc[[0,1,14,15],:]
M2 = df_point.loc[[1,2,3,17,16],:]
M3 = df_point.loc[[12,13,14,16,17],:]
M4 = df_point.loc[[3,4,11,12],:]
M5 = df_point.loc[[4,5,9,10,11],:]
M6 = df_point.loc[[5,6,35,9],:]
M7 = df_point.loc[[6,7,8,35],:]
E1 = df_point.loc[[18,19,20,21,22],:]
E2 = df_point.loc[[23,24,25,26],:]
E3 = df_point.loc[[27,28,29,30],:]
W1 = df_point.loc[[31],:]
T1 = df_point.loc[[32],:]
T2 = df_point.loc[[33],:]
T3 = df_point.loc[[34],:]
M1

jungbak = pd.read_csv('./통합데이터/df_train_v2.csv')
jungbak

# 버퍼 생성 코드
def create_buffer(lon, lat, radius):
    in_proj = Proj('epsg:4326')  # 입력 좌표 시스템 (WGS84)
    out_proj = Proj('epsg:3857')  # 출력 좌표 시스템 (WGS84 기반의 Web Mercator)

    # 좌표 변환: 위도, 경도 -> 메터 단위
    x, y = transform(in_proj, out_proj, lon, lat)
    point = Point(x, y)
    buffer = point.buffer(radius)
    # print('저는 버퍼입니다.', buffer)
    return buffer

#선회 반경 계산
def turning_raidus(lon, lat, area):
  radius = 0
  if area == 'E1' or area == 'T1':
    radius = 200
  elif area == 'E2' or area == 'W1':
    radius = 400
  elif area == 'E3':
    radius = 483
  elif area == 'T1':
    radius = 290
  elif area == 'T2' or area == 'T3':
    radius = 95
  else:
    radius = 90
  trans_buffer = create_buffer(lat, lon, radius)
  return trans_buffer


from shapely.geometry import Point, Polygon
from shapely.ops import cascaded_union

M1 = df_point.loc[[0,1,14,15],:]
M2 = df_point.loc[[1,2,3,17,16],:]
M3 = df_point.loc[[12,13,14,16,17],:]
M4 = df_point.loc[[3,4,11,12],:]
M5 = df_point.loc[[4,5,9,10,11],:]
M6 = df_point.loc[[5,6,35,9],:]
M7 = df_point.loc[[6,7,8,35],:]
E1 = df_point.loc[[18,19,20,21,22],:]
E2 = df_point.loc[[23,24,25,26],:]
E3 = df_point.loc[[27,28,29,30],:]
W1 = df_point.loc[[31],:]
T1 = df_point.loc[[32],:]
T2 = df_point.loc[[33],:]
T3 = df_point.loc[[34],:]

# W1_buffer = create_buffer(W1['경도'], W1['위도'], 400)
# T1_buffer = create_buffer(T1['경도'], T1['위도'], 300)
# T2_buffer = create_buffer(T2['경도'], T2['위도'], 300)
# T3_buffer = create_buffer(T3['경도'], T3['위도'], 250)

# W1_point = Point(W1['경도'].values[0], W1['위도'].values[0])
# T1_point = Point(T1['경도'].values[0], T1['위도'].values[0])
# T2_point = Point(T2['경도'].values[0], T2['위도'].values[0])
# T3_point = Point(T3['경도'].values[0], T3['위도'].values[0])

# W1_buffer = W1_point.buffer(W1_buffer)
# T1_buffer = T1_point.buffer(T1_buffer)
# T2_buffer = T2_point.buffer(T2_buffer)
# T3_buffer = T3_point.buffer(T3_buffer)

# 영역과 식별자를 매핑하는 딕셔너리 생성
areas = {
    'M1': cascaded_union([Polygon(M1)]),
    'M2': cascaded_union([Polygon(M2)]),
    'M3': cascaded_union([Polygon(M3)]),
    'M4': cascaded_union([Polygon(M4)]),
    'M5': cascaded_union([Polygon(M5)]),
    'M6': cascaded_union([Polygon(M6)]),
    'M7': cascaded_union([Polygon(M7)]),
    'E1': cascaded_union([Polygon(E1)]),
    'E2': cascaded_union([Polygon(E2)]),
    'E3': cascaded_union([Polygon(E3)]),
    # 따로 버퍼값 바꿔야됨
    # 'W1': cascaded_union([W1_buffer]),
    # 'T1': cascaded_union([T1_buffer]),
    # 'T2': cascaded_union([T2_buffer]),
    # 'T3': cascaded_union([T3_buffer])
}

# 테스트 좌표의 첫번재 값 가져옴
first_occurrences = jungbak.drop_duplicates(subset='num', keep='first')
first_occurrences
first_occurrences['num']
data = {
    'Ship Num': [],
    'Area': [],
    'Turning Radius': []
}
df = pd.DataFrame(data)

for idx, row in first_occurrences.iterrows():
  for area, shape in areas.items():
    # 데이터 들어오는 것에 따른 전처리
    # lon = float(row['lon'][2:-2])
    # lat = float(row['lat'][2:-2])
    lon = row['lon']
    lat = row['lat']
    #함수로 정박지와 선회반경 계산한
    turning_radius = turning_raidus(lon, lat, area)
    new_row = {'Ship Num': row['num'], 'Area': area, 'Turning Radius': turning_radius}
    df = df.append(new_row, ignore_index=True)
    
df.to_csv('./정박지 관련 데이터/area_turning_radius.csv',index=0)

C:\Users\sjo05\AppData\Local\Temp\ipykernel_26656\531000227.py:110: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  'M1': cascaded_union([Polygon(M1)]),
C:\Users\sjo05\AppData\Local\Temp\ipykernel_26656\531000227.py:111: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  'M2': cascaded_union([Polygon(M2)]),
C:\Users\sjo05\AppData\Local\Temp\ipykernel_26656\531000227.py:112: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  'M3': cascaded_union([Polygon(M3)]),
C:\Users\sjo05\AppData\Local\Temp\ipykernel_26656\531000227.py:113: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  'M4': cascaded_union([Polygon(M4)]),
C:\Users\sjo05\AppData\Local\Temp\ipykernel_26656\531000227.py:114: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()